# Customer Churn Prediction + Model Monitor
This notebook shows how to:
* Host a machine learning model in Amazon SageMaker and capture inference requests, results, and metadata 
* Analyze a training dataset to generate baseline constraints
* Monitor a live endpoint for violations against constraints

---
## Background

Amazon SageMaker provides every developer and data scientist with the ability to build, train, and deploy machine learning models quickly. Amazon SageMaker is a fully-managed service that encompasses the entire machine learning workflow. You can label and prepare your data, choose an algorithm, train a model, and then tune and optimize it for deployment. You can deploy your models to production with Amazon SageMaker to make predictions and lower costs than was previously possible.

In addition, Amazon SageMaker enables you to capture the input, output and metadata for invocations of the models that you deploy. It also enables you to analyze the data and monitor its quality. In this notebook, you learn how Amazon SageMaker enables these capabilities.


![model-monitor](./model-monitor.jpg)


### Imports

In [1]:
from sagemaker import get_execution_role, session
from sagemaker.model import Model
import sagemaker
import boto3
import json
import re
import os

### Essentials

In [2]:
session = sagemaker.Session()
bucket = session.default_bucket()
prefix = 'classifier/model-monitor'
# COPY the training job name of the previous training job you ran from the SageMaker Console under Training Jobs
# Let's re-use the Model we trained in Notebook 2 (Built-in Algorithm XGBoost)
training_job_name = 'classifier-2020-11-06-01-17-06-402' # CHANGE THIS
# COPY S3 model artifact location from the console
model_url = 's3://sagemaker-demo-892313895307/clf/model-artifacts/classifier-2020-11-06-01-17-06-402/output/model.tar.gz'
container_image_uri = '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3'
role = get_execution_role()

### S3 Paths

In [3]:
data_capture_prefix = '{}/datacapture'.format(prefix)
s3_capture_upload_path = 's3://{}/{}'.format(bucket, data_capture_prefix)

reports_prefix = '{}/reports'.format(prefix)
s3_report_path = 's3://{}/{}'.format(bucket,reports_prefix)

code_prefix = '{}/code'.format(prefix)

print("Capture path: {}".format(s3_capture_upload_path))
print("Report path: {}".format(s3_report_path))

Capture path: s3://sagemaker-us-east-1-892313895307/classifier/model-monitor/datacapture
Report path: s3://sagemaker-us-east-1-892313895307/classifier/model-monitor/reports


## Capture Real-Time Inference Data from SageMaker Endpoints

In [4]:
model = Model(image=container_image_uri, model_data=model_url, role=role)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


### Re-deploy using DataCaptureConfig

To enable data capture for monitoring the model data quality, you specify the new capture option called DataCaptureConfig. You can capture the request payload, the response payload or both with this configuration. The capture config applies to all variants. Go ahead with the deployment.

In [5]:
from sagemaker.model_monitor import DataCaptureConfig
from time import gmtime, strftime

In [6]:
endpoint_name = 'classifier-xgboost-model-monitor-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

data_capture_config = DataCaptureConfig(
                        enable_capture=True,
                        sampling_percentage=100,
                        destination_s3_uri=s3_capture_upload_path)

EndpointName=classifier-xgboost-model-monitor-2020-11-08-01-54-47


In [7]:
data_capture_config.__dict__

{'enable_capture': True,
 'sampling_percentage': 100,
 'destination_s3_uri': 's3://sagemaker-us-east-1-892313895307/classifier/model-monitor/datacapture',
 'kms_key_id': None,
 'capture_options': ['REQUEST', 'RESPONSE'],
 'csv_content_types': ['text/csv'],
 'json_content_types': ['application/json']}

In [8]:
model.deploy(initial_instance_count=1, 
             instance_type='ml.m5.xlarge', 
             endpoint_name=endpoint_name, 
             data_capture_config=data_capture_config)

-------------!

### Invoke the Deployed Model Monitor Endpoint 


You can now send data to this endpoint to get inferences in real time. Because you enabled the data capture in the previous steps, the request and response payload, along with some additional metadata, is saved in the Amazon Simple Storage Service (Amazon S3) location you have specified in the DataCaptureConfig.

This step invokes the endpoint with included sample data for about 2 minutes. Data is captured based on the sampling percentage specified and the capture continues until the data capture option is turned off.

In [18]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.predictor import csv_serializer

import pandas as pd
pd.set_option('display.max_colwidth', -1)

import numpy as np
import json
import time

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [19]:
predictor = RealTimePredictor(endpoint=endpoint_name)

In [20]:
predictor.content_type = 'text/csv'
predictor.serializer = csv_serializer

In [21]:
test_df = pd.read_csv('.././DATA/test/test.csv', names=['class', 'mass', 'width', 'height', 'color_score'])

In [22]:
test_df.head()

,class,mass,width,height,color_score
0,1,0.142857,0.058824,0.538462,0.382353
1,3,0.371429,0.529412,0.646154,0.588235
2,0,0.314286,0.441176,0.569231,0.323529
3,1,0.157143,0.058824,0.676923,0.441176
4,3,0.457143,0.500000,0.800000,0.529412


In [23]:
for _, row in test_df.iterrows():
    X =[row.mass, row.width, row.height, row.color_score]
    payload = np.array(X)
    response = predictor.predict(data=payload)
    print(response.decode('utf-8'))

1.0
3.0
1.0
3.0
3.0
3.0
1.0
3.0
0.0
0.0
3.0
3.0
0.0
0.0
2.0


### View captured data

NOTE: Could take a minute here

Now list the data capture files stored in Amazon S3. You should expect to see different files from different time periods organized based on the hour in which the invocation occurred. The format of the Amazon S3 path is:

`s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`

In [24]:
s3_client = boto3.Session().client('s3')
current_endpoint_capture_prefix = '{}/{}'.format(data_capture_prefix, endpoint_name)
current_endpoint_capture_prefix

'classifier/model-monitor/datacapture/classifier-xgboost-model-monitor-2020-11-08-01-54-47'

In [25]:
result = s3_client.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
result.get('Contents')

[{'Key': 'classifier/model-monitor/datacapture/classifier-xgboost-model-monitor-2020-11-08-01-54-47/AllTraffic/2020/11/08/02/01-24-794-3a684b44-17c4-4003-bfbe-caacf1be0120.jsonl',
  'LastModified': datetime.datetime(2020, 11, 8, 2, 2, 29, tzinfo=tzlocal()),
  'ETag': '"103515d10cc84349a6562fe59f26409a"',
  'Size': 6326,
  'StorageClass': 'STANDARD'}]

In [26]:
capture_files = [capture_file.get("Key") for capture_file in result.get('Contents')]
print("Found Capture Files:")
print("\n ".join(capture_files))

Found Capture Files:
classifier/model-monitor/datacapture/classifier-xgboost-model-monitor-2020-11-08-01-54-47/AllTraffic/2020/11/08/02/01-24-794-3a684b44-17c4-4003-bfbe-caacf1be0120.jsonl


Next, view the contents of a single capture file. Here you should see all the data captured in an Amazon SageMaker specific JSON-line formatted file. Take a quick peek at the first few lines in the captured file.

In [27]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get('Body').read().decode("utf-8")

In [28]:
capture_file = get_obj_body(capture_files[-1])
capture_file[:1000]

'{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"0.14285714285714285,0.058823529411764726,0.5384615384615385,0.38235294117647056","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"1.0","encoding":"CSV"}},"eventMetadata":{"eventId":"cf0945fc-0971-470a-9903-639f1122781c","inferenceTime":"2020-11-08T02:01:24Z"},"eventVersion":"0"}\n{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"0.3714285714285714,0.5294117647058825,0.6461538461538461,0.5882352941176472","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"3.0","encoding":"CSV"}},"eventMetadata":{"eventId":"1960cc16-c0bd-494d-bc9f-f6b602b8947d","inferenceTime":"2020-11-08T02:01:24Z"},"eventVersion":"0"}\n{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"0.31428571428571433,0.4411764705882353,0.569230769230769

Finally, the contents of a single line is present below in a formatted JSON file so that you can observe a little better.

In [29]:
print(json.dumps(json.loads(capture_file.split('\n')[0]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "0.14285714285714285,0.058823529411764726,0.5384615384615385,0.38235294117647056",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "1.0",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "cf0945fc-0971-470a-9903-639f1122781c",
    "inferenceTime": "2020-11-08T02:01:24Z"
  },
  "eventVersion": "0"
}


As you can see, each inference request is captured in one line in the jsonl file. The line contains both the input and output merged together. In the example, you provided the ContentType as `text/csv` which is reflected in the `observedContentType` value. Also, you expose the encoding that you used to encode the input and output payloads in the capture format with the `encoding` value.

To recap, you observed how you can enable capturing the input or output payloads to an endpoint with a new parameter. You have also observed what the captured format looks like in Amazon S3. Next, continue to explore how Amazon SageMaker helps with monitoring the data collected in Amazon S3.

## Baseling & Continuous Monitoring

In addition to collecting the data, Amazon SageMaker provides the capability for you to monitor and evaluate the data observed by the endpoints. For this:
1. Create a baseline with which you compare the realtime traffic. 
1. Once a baseline is ready, setup a schedule to continously evaluate and compare against the baseline.

The training dataset with which you trained the model is usually a good baseline dataset. Note that the training dataset data schema and the inference dataset schema should exactly match (i.e. the number and order of the features).

From the training dataset you can ask Amazon SageMaker to suggest a set of baseline `constraints` and generate descriptive `statistics` to explore the data. For this example, upload the training dataset that was used to train the pre-trained model included in this example. If you already have it in Amazon S3, you can directly point to it.

In [30]:
# copy over the training dataset to Amazon S3 (if you already have it in Amazon S3, you could reuse it)
baseline_prefix = prefix + '/baselining'
baseline_data_prefix = baseline_prefix + '/data'
baseline_results_prefix = baseline_prefix + '/results'

baseline_data_uri = 's3://{}/{}'.format(bucket,baseline_data_prefix)
baseline_results_uri = 's3://{}/{}'.format(bucket, baseline_results_prefix)
print('Baseline data uri: {}'.format(baseline_data_uri))
print('Baseline results uri: {}'.format(baseline_results_uri))

Baseline data uri: s3://sagemaker-us-east-1-892313895307/classifier/model-monitor/baselining/data
Baseline results uri: s3://sagemaker-us-east-1-892313895307/classifier/model-monitor/baselining/results


### Create a Baselining Job with Training Dataset

In [31]:
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker.model_monitor import DefaultModelMonitor

#### Upload Train Set to S3 as Baseline Data

In [32]:
train_data = open('.././DATA/train/train_with_header.csv', 'rb')
s3_key = os.path.join(baseline_prefix, 'data', 'train_with_header.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(s3_key).upload_fileobj(train_data)

In [33]:
default_model_monitor = DefaultModelMonitor(
                            role=role,
                            instance_count=1,
                            instance_type='ml.r5.xlarge',
                            volume_size_in_gb=20,
                            max_runtime_in_seconds=3600,
                        )

In [34]:
default_model_monitor.suggest_baseline(
                            baseline_dataset=baseline_data_uri + '/train_with_header.csv',
                            dataset_format=DatasetFormat.csv(header=True),
                            output_s3_uri=baseline_results_uri,
                            wait=True
                        )


Job Name:  baseline-suggestion-job-2020-11-08-02-19-12-931
Inputs:  [{'InputName': 'baseline_dataset_input', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-892313895307/classifier/model-monitor/baselining/data/train_with_header.csv', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-892313895307/classifier/model-monitor/baselining/results', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
........................2020-11-08 02:23:01,577 - __main__ - INFO - All params:{'ProcessingJobArn': 'arn:aws:sagemaker:us-east-1:892313895307:processing-job/baseline-suggestion-job-2020-11-08-02-19-12-931', 'ProcessingJobName': 'baseline-suggestion-job-2020-11-08-02-19-12-931', 'Environment': {'dataset_format': '{"csv": {"header": true, "output_col

2020-11-08 02:23:13,394 - bootstrap - INFO - Failed to run /usr/hadoop-3.0.0/bin/yarn --daemon start proxyserver, return code 1
2020-11-08 02:23:13,394 - DefaultDataAnalyzer - INFO - Total number of hosts in the cluster: 1
2020-11-08 02:23:23,404 - DefaultDataAnalyzer - INFO - Running command: bin/spark-submit --master yarn --deploy-mode client --conf spark.hadoop.fs.s3a.aws.credentials.provider=org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider --conf spark.serializer=org.apache.spark.serializer.KryoSerializer /opt/amazon/sagemaker-data-analyzer-1.0-jar-with-dependencies.jar --analytics_input /tmp/spark_job_config.json
2020-11-08 02:23:24 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2020-11-08 02:23:24 INFO  Main:27 - Start analyzing with args: --analytics_input /tmp/spark_job_config.json
2020-11-08 02:23:24 INFO  Main:30 - Analytics input path: DataAnalyzerParams(/tmp/spark_job_config.json,yarn)
2

2020-11-08 02:23:55 INFO  YarnClientSchedulerBackend:54 - SchedulerBackend is ready for scheduling beginning after waiting maxRegisteredResourcesWaitingTime: 30000(ms)
2020-11-08 02:23:55 WARN  SparkContext:66 - Spark is not running in local mode, therefore the checkpoint directory must not be on the local filesystem. Directory '/tmp' appears to be on the local filesystem.
2020-11-08 02:23:55 INFO  DatasetReader:90 - Files to process:List(file:///opt/ml/processing/input/baseline_dataset_input/train_with_header.csv)
2020-11-08 02:23:55 INFO  SharedState:54 - Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir ('file:/usr/spark-2.3.1/spark-warehouse').
2020-11-08 02:23:55 INFO  SharedState:54 - Warehouse path is 'file:/usr/spark-2.3.1/spark-warehouse'.
2020-11-08 02:23:55 INFO  StateStoreCoordinatorRef:54 - Registered StateStoreCoordinator endpoint
2020-11-08 02:23:56 INFO  FileSourceStrategy:54 - Pruning directories with: 
2020-11-08 02:23:56 INFO  File


2020-11-08 02:24:00 INFO  BlockManagerInfo:54 - Added rdd_11_0 in memory on algo-1:43055 (size: 3.5 KB, free: 11.9 GB)
2020-11-08 02:24:00 INFO  TaskSetManager:54 - Finished task 0.0 in stage 1.0 (TID 1) in 371 ms on algo-1 (executor 1) (1/1)
2020-11-08 02:24:00 INFO  DAGScheduler:54 - ResultStage 1 (head at DataAnalyzer.scala:79) finished in 0.408 s
2020-11-08 02:24:00 INFO  YarnScheduler:54 - Removed TaskSet 1.0, whose tasks have all completed, from pool 
2020-11-08 02:24:00 INFO  DAGScheduler:54 - Job 1 finished: head at DataAnalyzer.scala:79, took 0.413118 s
2020-11-08 02:24:00 WARN  Utils:66 - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.
2020-11-08 02:24:00 INFO  CodeGenerator:54 - Code generated in 18.337049 ms
2020-11-08 02:24:00 INFO  SparkContext:54 - Starting job: collect at AnalysisRunner.scala:313
2020-11-08 02:24:00 INFO  DAGScheduler:54 - Registering RDD 22

### Explore the generated constraints and statistics

In [35]:
s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=bucket, Prefix=baseline_results_prefix)
report_files = [report_file.get('Key') for report_file in result.get('Contents')]

In [36]:
print('Found Files:')
print("\n ".join(report_files))

Found Files:
classifier/model-monitor/baselining/results/constraints.json
 classifier/model-monitor/baselining/results/statistics.json


In [37]:
baseline_job = default_model_monitor.latest_baselining_job

In [38]:
baseline_job.__dict__

{'inputs': [<sagemaker.processing.ProcessingInput at 0x7fbd3cb52588>],
 'outputs': [<sagemaker.processing.ProcessingOutput at 0x7fbd3cb525f8>],
 'output_kms_key': None,
 'sagemaker_session': <sagemaker.session.Session at 0x7fbd3cb945f8>,
 'job_name': 'baseline-suggestion-job-2020-11-08-02-19-12-931'}

In [39]:
baseline_job.suggested_constraints().body_dict['monitoring_config']

Parameter 'session' will be renamed to 'sagemaker_session' in SageMaker Python SDK v2.


{'evaluate_constraints': 'Enabled',
 'emit_metrics': 'Enabled',
 'datatype_check_threshold': 1.0,
 'domain_content_threshold': 1.0,
 'distribution_constraints': {'perform_comparison': 'Enabled',
  'comparison_threshold': 0.1,
  'comparison_method': 'Robust'}}

In [40]:
schema_df = pd.io.json.json_normalize(baseline_job.baseline_statistics().body_dict['features'])
schema_df.head(10)

Parameter 'session' will be renamed to 'sagemaker_session' in SageMaker Python SDK v2.
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


,name,inferred_type,numerical_statistics.common.num_present,numerical_statistics.common.num_missing,numerical_statistics.mean,numerical_statistics.sum,numerical_statistics.std_dev,numerical_statistics.min,numerical_statistics.max,numerical_statistics.distribution.kll.buckets,numerical_statistics.distribution.kll.sketch.parameters.c,numerical_statistics.distribution.kll.sketch.parameters.k,numerical_statistics.distribution.kll.sketch.data
0,mass,Fractional,44,0,0.301948,13.285714,0.184394,0.0,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'count': 4.0}, {'lower_bound': 0.1, 'upper_bound': 0.2, 'count': 5.0}, {'lower_bound': 0.2, 'upper_bound': 0.3, 'count': 15.0}, {'lower_bound': 0.3, 'upper_bound': 0.4, 'count': 13.0}, {'lower_bound': 0.4, 'upper_bound': 0.5, 'count': 5.0}, {'lower_bound': 0.5, 'upper_bound': 0.6, 'count': 0.0}, {'lower_bound': 0.6, 'upper_bound': 0.7, 'count': 0.0}, {'lower_bound': 0.7, 'upper_bound': 0.8, 'count': 0.0}, {'lower_bound': 0.8, 'upper_bound': 0.9, 'count': 0.0}, {'lower_bound': 0.9, 'upper_bound': 1.0, 'count': 2.0}]",0.64,2048.0,"[[0.32142857142857145, 0.42142857142857143, 0.36428571428571427, 0.2785714285714285, 0.19285714285714284, 0.2, 0.28571428571428575, 0.3714285714285714, 0.01428571428571429, 0.24285714285714283, 0.2785714285714285, 0.49999999999999994, 0.0, 0.14285714285714285, 0.3, 0.95, 0.28571428571428575, 0.14285714285714285, 0.2642857142857143, 0.3, 0.2714285714285714, 0.0357142857142857, 0.02857142857142858, 0.28571428571428575, 0.4428571428571429, 0.3, 0.2714285714285714, 0.15000000000000002, 1.0, 0.15000000000000002, 0.35000000000000003, 0.3428571428571428, 0.42857142857142855, 0.31428571428571433, 0.4071428571428572, 0.2285714285714286, 0.2785714285714285, 0.3071428571428571, 0.32857142857142857, 0.29285714285714287, 0.2357142857142857, 0.2285714285714286, 0.35714285714285715, 0.39285714285714285]]"
1,width,Fractional,44,0,0.370989,16.323529,0.216151,0.0,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.10000000000000002, 'count': 8.0}, {'lower_bound': 0.10000000000000002, 'upper_bound': 0.20000000000000004, 'count': 2.0}, {'lower_bound': 0.20000000000000004, 'upper_bound': 0.3000000000000001, 'count': 3.0}, {'lower_bound': 0.3000000000000001, 'upper_bound': 0.4000000000000001, 'count': 7.0}, {'lower_bound': 0.4000000000000001, 'upper_bound': 0.5000000000000001, 'count': 14.0}, {'lower_bound': 0.5000000000000001, 'upper_bound': 0.6000000000000002, 'count': 7.0}, {'lower_bound': 0.6000000000000002, 'upper_bound': 0.7000000000000002, 'count': 1.0}, {'lower_bound': 0.7000000000000002, 'upper_bound': 0.8000000000000002, 'count': 0.0}, {'lower_bound': 0.8000000000000002, 'upper_bound': 0.9000000000000001, 'count': 0.0}, {'lower_bound': 0.9000000000000001, 'upper_bound': 1.0000000000000002, 'count': 2.0}]",0.64,2048.0,"[[0.32352941176470584, 0.41176470588235303, 0.3823529411764708, 0.35294117647058854, 0.05882352941176472, 0.0, 0.470588235294118, 0.6470588235294119, 0.02941176470588247, 0.2941176470588236, 0.3823529411764708, 0.4411764705882353, 0.0, 0.08823529411764697, 0.35294117647058854, 0.9411764705882353, 0.5588235294117647, 0.1470588235294117, 0.3823529411764708, 0.5000000000000002, 0.2058823529411764, 0.11764705882352944, 0.05882352941176472, 0.5294117647058825, 0.4411764705882353, 0.3823529411764708, 0.5294117647058825, 0.02941176470588247, 1.0000000000000002, 0.08823529411764697, 0.4411764705882353, 0.470588235294118, 0.4411764705882353, 0.41176470588235303, 0.5000000000000002, 0.4411764705882353, 0.41176470588235303, 0.470588235294118, 0.5000000000000002, 0.41176470588235303, 0.5294117647058825, 0.26470588235294135, 0.470588235294118, 0.41176470588235303]]"
2,height,Fractional,44,0,0.568531,25.015385,0.219515,0.0,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'count': 3.0}, {'lower_bound': 0.1, 'upper_bound': 0.2, 'count': 1.0}, {'lower_bound': 0.2, 'upper_bound': 0.3, 'count': 0.0}, {'lower_bound': 0.3, 'upper_bound': 0.4, 'count': 0.0}, {'lower_bound': 0.4, 'upper_bound': 0.5,

In [41]:
constraints_df = pd.io.json.json_normalize(baseline_job.suggested_constraints().body_dict['features'])
constraints_df.head(10)

Parameter 'session' will be renamed to 'sagemaker_session' in SageMaker Python SDK v2.
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


,name,inferred_type,completeness,num_constraints.is_non_negative
0,mass,Fractional,1.0,True
1,width,Fractional,1.0,True
2,height,Fractional,1.0,True
3,color_score,Fractional,1.0,True


## Monitoring Schedules

<p><b>Analyzing collected data for data quality issues</b></p>

When you have collected the data above, analyze and monitor the data with Monitoring Schedules

### Create a Schedule

In [42]:
from sagemaker.model_monitor import CronExpressionGenerator
from time import gmtime, strftime

In [43]:
mon_schedule_name = 'clf-xgb-model-monitor-schedule-' + strftime('%Y-%m-%d-%H-%M-%S', gmtime())

default_model_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_name,
    endpoint_input=predictor.endpoint,
    output_s3_uri=s3_report_path,
    statistics=default_model_monitor.baseline_statistics(),
    constraints=default_model_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

Parameter 'session' will be renamed to 'sagemaker_session' in SageMaker Python SDK v2.
Parameter 'session' will be renamed to 'sagemaker_session' in SageMaker Python SDK v2.



Creating Monitoring Schedule with name: clf-xgb-model-monitor-schedule-2020-11-08-02-26-29


### Test Model Monitoring using Artificial Traffic
The cell below starts a thread to send some traffic to the endpoint. Note that you need to stop the kernel to terminate this thread. If there is no traffic, the monitoring jobs are marked as `Failed` since there is no data to process.

In [44]:
from threading import Thread
from time import sleep
import time

In [45]:
# endpoint_name = predictor.endpoint
endpoint_name = 'classifier-xgboost-model-monitor-2020-11-08-01-54-47'
runtime_client = boto3.client('runtime.sagemaker')

In [46]:
def invoke_endpoint(endpoint_name, file_name, runtime_client):
    with open(file_name, 'r') as f:
        for row in f:
            payload = row.rstrip('\n')
            runtime_client.invoke_endpoint(EndpointName=endpoint_name, 
                                           ContentType='text/csv', 
                                           Body=payload)
            #prediction = response['Body'].read()

In [47]:
def invoke_endpoint_forever():
    while True:
        invoke_endpoint(endpoint_name, '.././DATA/test/model_monitor_test.csv', runtime_client)

In [48]:
thread = Thread(target = invoke_endpoint_forever)
thread.start()
# NOTE: You need to stop the kernel to stop the invocations

### Describe and Inspect the Schedule
Once you describe, observe that the MonitoringScheduleStatus changes to Scheduled.

In [49]:
desc_schedule_result = default_model_monitor.describe_schedule()
print('Schedule status: {}'.format(desc_schedule_result['MonitoringScheduleStatus']))

Schedule status: Scheduled


### List executions
The schedule starts jobs at the previously specified intervals. Here, you list the latest five executions. Note that if you are kicking this off after creating the hourly schedule, you might find the executions empty. You might have to wait until you cross the hour boundary (in UTC) to see executions kick off. The code below has the logic for waiting.

Note: Even for an hourly schedule, Amazon SageMaker has a buffer period of 20 minutes to schedule your execution. You might see your execution start in anywhere from zero to ~20 minutes from the hour boundary. This is expected and done for load balancing in the backend.

In [64]:
mon_executions = default_model_monitor.list_executions()

print("We created a hourly schedule above and it will kick off executions ON the hour (plus 0 - 20 min buffer).")
print("We will have to wait till we hit the hour ...") 



We created a hourly schedule above and it will kick off executions ON the hour (plus 0 - 20 min buffer).
We will have to wait till we hit the hour ...


In [65]:
# Wait till you see an execution object in this list before you proceed to the next step
# takes between (60 to 80 mins)
mon_executions

### Inspect a specific execution (latest execution)
In the previous cell, you picked up the latest completed or failed scheduled execution. Here are the possible terminal states and what each of them mean: 
* Completed - This means the monitoring execution completed and no issues were found in the violations report.
* CompletedWithViolations - This means the execution completed, but constraint violations were detected.
* Failed - The monitoring execution failed, maybe due to client error (perhaps incorrect role premissions) or infrastructure issues. Further examination of FailureReason and ExitMessage is necessary to identify what exactly happened.
* Stopped - Job exceeded max runtime or was manually stopped.

In [68]:
latest_execution = mon_executions[-1] # latest execution's index is -1, second to last is -2 and so on ...
latest_execution.describe()

{'ProcessingInputs': [{'InputName': 'input_1',
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-892313895307/classifier/model-monitor/datacapture/classifier-xgboost-model-monitor-2020-11-08-01-54-47/AllTraffic/2020/11/08/02',
    'LocalPath': '/opt/ml/processing/input/endpoint/classifier-xgboost-model-monitor-2020-11-08-01-54-47/AllTraffic/2020/11/08/02',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}},
  {'InputName': 'baseline',
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-892313895307/classifier/model-monitor/baselining/results/statistics.json',
    'LocalPath': '/opt/ml/processing/baseline/stats',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated'}},
  {'InputName': 'constraints',
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-892313895307/classifier/model-monitor/baselining/results/constraints.json',
    'LocalPath': '/opt/ml/pr

In [71]:
print("Latest Execution Status: {}".format(latest_execution.describe()['ProcessingJobStatus']))
print("Latest Execution Result: {}".format(latest_execution.describe()['ExitMessage']))

Latest Execution Status: Completed
Latest Execution Result: CompletedWithViolations: Job completed successfully with 1 violations.


In [72]:
report_uri = latest_execution.output.destination
print('Report URI: {}'.format(report_uri))

Report URI: s3://sagemaker-us-east-1-892313895307/classifier/model-monitor/reports/classifier-xgboost-model-monitor-2020-11-08-01-54-47/clf-xgb-model-monitor-schedule-2020-11-08-02-26-29/2020/11/08/03


### List the Generated Reports

In [73]:
from urllib.parse import urlparse

In [74]:
s3uri = urlparse(report_uri)
report_bucket = s3uri.netloc
report_key = s3uri.path.lstrip('/')
print('Report Bucket: {}'.format(report_bucket))
print('Report Key: {}'.format(report_key))

Report Bucket: sagemaker-us-east-1-892313895307
Report Key: classifier/model-monitor/reports/classifier-xgboost-model-monitor-2020-11-08-01-54-47/clf-xgb-model-monitor-schedule-2020-11-08-02-26-29/2020/11/08/03


In [75]:
s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=report_bucket, Prefix=report_key)
report_files = [report_file.get("Key") for report_file in result.get('Contents')]
print("Found Report Files:")
print("\n ".join(report_files))

Found Report Files:
classifier/model-monitor/reports/classifier-xgboost-model-monitor-2020-11-08-01-54-47/clf-xgb-model-monitor-schedule-2020-11-08-02-26-29/2020/11/08/03/constraint_violations.json


### Violations Report

If there are any violations compared to the baseline, they will be listed here.

In [76]:
violations = default_model_monitor.latest_monitoring_constraint_violations()
violations.__dict__

Parameter 'session' will be renamed to 'sagemaker_session' in SageMaker Python SDK v2.


{'body_dict': {'violations': [{'feature_name': 'Extra columns',
    'constraint_check_type': 'extra_column_check',
    'description': 'There are extra columns in current dataset. Number of columns in current dataset: 5, Number of columns in baseline constraints: 4'}]},
 'file_s3_uri': 's3://sagemaker-us-east-1-892313895307/classifier/model-monitor/reports/classifier-xgboost-model-monitor-2020-11-08-01-54-47/clf-xgb-model-monitor-schedule-2020-11-08-02-26-29/2020/11/08/03/constraint_violations.json',
 'kms_key': None,
 'session': None}

In [77]:
constraints_df = pd.json_normalize(violations.body_dict["violations"])

In [78]:
constraints_df

,feature_name,constraint_check_type,description
0,Extra columns,extra_column_check,"There are extra columns in current dataset. Number of columns in current dataset: 5, Number of columns in baseline constraints: 4"


### Shift in Statistics 

In [ ]:
violations = default_model_monitor.latest_monitoring_constraint_violations()

In [ ]:
statistics_shift = default_model_monitor.latest_monitoring_statistics()
statistics_shift.__dict__

### Other Commands
We can also start and stop the monitoring schedules.

In [ ]:
# my_default_monitor.stop_monitoring_schedule()
# my_default_monitor.start_monitoring_schedule()

## Delete the Resources

You can keep your endpoint running to continue capturing data. If you do not plan to collect more data or use this endpoint further, you should delete the endpoint to avoid incurring additional charges. Note that deleting your endpoint does not delete the data that was captured during the model invocations. That data persists in Amazon S3 until you delete it yourself.

But before that, you need to delete the schedule first.

In [ ]:
# my_default_monitor.delete_monitoring_schedule()
# time.sleep(60) # actually wait for the deletion

In [ ]:
# predictor.delete_endpoint()

In [ ]:
# predictor.delete_model()